# Derivation of Time Complexity for Bubble Sort

Bubble Sort works by repeatedly comparing adjacent elements and swapping them if they are in the wrong order. The derivation of its time complexity involves analyzing how many comparisons and swaps are performed in the worst case.

## Step-by-Step Derivation

### 1. Understand the Algorithm
For an array of size `n`:

- In the first pass, the algorithm compares `n - 1` adjacent pairs.
- In the second pass, it compares `n - 2` pairs (because the largest element is already in its final position).
- This process continues until the last pass, which requires only 1 comparison.

### 2. Number of Comparisons
The total number of comparisons is the sum of the comparisons made in each pass:



C(n) = (n - 1) + (n - 2) + (n - 3) + ... + 1


This is an arithmetic series with:

- First term `a = 1`
- Last term `l = n - 1`
- Number of terms `k = n - 1`

The sum of an arithmetic series is:



S = k / 2 * (a + l)


Substituting the values:



C(n) = (n - 1) / 2 * (1 + (n - 1)) = (n - 1) / 2 * n


Thus:



C(n) = n * (n - 1) / 2


For large `n`, the dominant term is `n^2`, so:



C(n) ∈ O(n^2)

### 3. Number of Swaps
In the worst case (when the array is sorted in reverse order), every comparison results in a swap. Therefore, the number of swaps is the same as the number of comparisons:

S(n) = n * (n - 1) / 2 ∈ O(n^2)

### 4. Overall Time Complexity
The time complexity of Bubble Sort is determined by the number of comparisons and swaps. Both are `O(n^2)`. Thus, the overall time complexity is:

T(n) ∈ O(n^2)

# Overall Time Complexity of the MPI-Driven Bubble Sort:
- Splitting and Broadcasting: The broadcasting and splitting of the data are parallel operations. The broadcasting has a complexity of `O(n)` for all processes, and splitting the data into `8` chunks takes `O(n)` time in total.

- Sorting Each Chunk (Parallel Step): Each of the 8 processes sorts one chunk of size `n / 8`. The time complexity for sorting one chunk using bubble sort is `O((n / 8)^2)`. Since there are 8 processes working in parallel, the total time for sorting the chunks is `O((n / 8)^2)` per process. Since all processes are running in parallel, we don't multiply by `8` here, but we take the time complexity for the worst-case scenario, which is `O((n / 8)^2)`.

- Gathering Sorted Chunks: The gathering operation has a time complexity of `O(n)` because we are sending `n` elements back to the root process.

- Final Sorting on Root Process: The final sorting of the combined chunks on the root process has a time complexity of `O(n^2)` because it sorts the entire dataset.

## Final Time Complexity:
- Parallel Sorting: The parallel sorting on each chunk takes `O((n / 8)^2) = O(n^2 / 64)`. However, since this is parallelized across 8 processes, we focus on the slowest operation in the pipeline, which will be the final sorting step.

- Final Sorting on Root: The final sorting of the entire dataset has a time complexity of `O(n^2)`.

- Thus, the overall time complexity of the MPI-driven bubble sort is dominated by the final sorting on the root process, which is `O(n^2)`.

## Summary of Time Complexity:
- Broadcasting the data: `O(n)`
- Splitting the data into chunks: `O(n)`
- Sorting each chunk (in parallel): `O(n^2 / 64)` per process (but since it’s parallel, we don't multiply this by 8 - processes, so it's just `O(n^2 / 64))`
- Gathering the sorted chunks: `O(n)`
- Final sorting on root: `O(n^2)`

# Understanding Why MPI-based Bubble Sort is Faster Despite Same Time Complexity

The key point here is that while the **asymptotic time complexity** of the two approaches (bubble sort with and without MPI) is the same — \( O(n^2) \) — the actual performance of the MPI-based approach is faster due to the parallelization and other factors that affect real-world performance. Let's break down why this happens:

## 1. Parallelization and Speedup
In the MPI-based approach, the dataset is split into **8 chunks** (since you have 8 processes), and each chunk is sorted in parallel by a separate process.

- The sorting of chunks is done simultaneously, meaning that while one process is sorting its chunk, others are doing the same. This results in a **reduction** in the effective time it takes to sort the chunks, as compared to the non-parallelized bubble sort that processes the entire array sequentially.

### Without MPI (sequential):
- The bubble sort runs in a **single process** with a time complexity of \( O(n^2) \).

### With MPI (parallel):
- The sorting of chunks is done in parallel across 8 processes, so the **effective time complexity** for the sorting portion is reduced to approximately \( O\left(\frac{n^2}{64}\right) \), as each process only sorts \( \frac{n}{8} \) elements.
- The parallelism effectively divides the workload among multiple processes, which reduces the time taken to sort the data.

## 2. Communication Overhead
- **Communication**: There is an overhead associated with communication between processes, specifically during the `bcast` (broadcasting the data to all processes) and `gather` (collecting the sorted chunks to the root process) steps. This overhead depends on factors like **network latency** and the efficiency of MPI communication.
- However, in your case, the **communication overhead** is likely smaller compared to the time saved from parallel processing. Since you're using 8 processes, the **time savings** from parallelism outweigh the communication costs in this case.

## 3. Chunk Size and Workload Division
- In the MPI approach, each process is sorting a smaller **chunk** of the dataset. For example, if you have 10,000 elements and 8 processes, each process sorts around **1,250 elements**.
    - Sorting a smaller chunk with bubble sort \( O(k^2) \) where \( k \) is the chunk size will naturally be faster than sorting the entire array of size \( n \).
    - The parallel sorting of 8 smaller chunks leads to a **reduction in execution time**, especially in real-world scenarios where there are multiple cores/processors at work.

## 4. Caching and Memory Access
- With parallelization, there can also be better **memory caching** due to each process working on its local chunk. When sorting a smaller set of data, the **CPU cache** is more efficient, leading to faster sorting within each process.
- **Sequential sorting** (without MPI) might face cache misses or slower memory access since all the data is in one place and the memory accesses are not as localized.

## 5. Overall Performance
Although the theoretical time complexity of both versions of the algorithm is \( O(n^2) \), in practice, parallelization results in **faster execution** because:
- The workload is **distributed** across multiple cores.
- Sorting a smaller subset of data is faster in terms of both **computation** and **memory access**.
- **Communication overheads** in MPI are often outweighed by the gains from parallel processing.

## Key Takeaways:
- **MPI-based approach**: The speedup you're seeing is due to the parallel sorting of chunks, which reduces the total time spent on sorting.
- **Sequential approach**: In contrast, the non-parallel (sequential) bubble sort works on the entire array in a single thread, which results in higher execution time despite having the same \( O(n^2) \) time complexity.

## Why is MPI Faster Despite the Same Time Complexity?
- **Parallel processing** divides the work into smaller tasks that can be processed simultaneously, so while the theoretical time complexity is the same, the **practical execution time** is reduced.
- In a real system, with **8 cores** and **8 processes**, each core works on a different chunk of data, and the **total processing time** is much lower than in the sequential case.

Thus, **MPI-driven parallelization** gives you significant **practical speedup** even if the theoretical time complexity (asymptotic behavior) remains the same.
